In [2]:
#Part 1: Export free recalls grouped by participant
#Part 2: Export free recalls grouped by story
#Part 3: convert each set of story recalls into a scorable excel sheet

In [22]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
import os
import glob
import math
import statistics
import matplotlib.pyplot as plt
import pickle
%autosave 5

plotly.tools.set_credentials_file(username='amr7', api_key='4gj9vxlD7C63cCRCaRdU')

#iMac
#directory_stories = '/Users/alexreblando/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/ebs/story_xlsx_files'

#laptop
directory = '/Users/alexreblando/Documents/GitHub/ebs/behavioral_August2019/psychopy experiment/usable_data'
directory_stories = '/Users/alexreblando/Documents/GitHub/ebs/behavioral_August2019/story_xlsx_files_RECORDED'
filenames = glob.glob(directory+ '/*.csv')
filenames_stories = glob.glob(directory_stories + '/*.xlsx')

dfs = []
dfs_stories = []
    
for filename in filenames:
    dfs.append(pd.read_csv(filename))
    
for filename in filenames_stories:
    dfs_stories.append(pd.read_excel(filename))
    
print('done!')

Autosaving every 5 seconds
done!


In [4]:
#Part 1: Export free recalls grouped by participant

In [6]:
#create a dictionary for each participant that contains which stories they read, whether they were paired with the
#social or location schema, and the free recall for each story

size_dfs = len(dfs)
participants = dict()

for s in range(size_dfs):
    this_dict = dict()
    check = 0
    this_p = dfs[s]['participant'].iloc[0]
    story_list = dfs[s]['order of stories'].iloc[0].split(' ')
    story_list = [elem.replace("[","").replace("]","") for elem in story_list]
    schema_list = dfs[s]['order of perspectives'].iloc[0].split(' ')
    schema_list = [elem.replace("[","").replace("]","") for elem in schema_list]
    schema_list = [elem.replace("\n","") for elem in schema_list]
    schema_list = [elem.replace("'","") for elem in schema_list]
    #extract all elements in 'recall' column
    h = dfs[s]['recall'].notnull()
    recall_list = np.zeros(8)
    recall_list =recall_list.tolist()
    for i in range(len(h)):
        if h[i] == True:
            if dfs[s]['count'].iloc[i] == 0:
                if check == 0:
                    recall_list[0] = dfs[s]['recall'].iloc[i]
                    check = 1
                else:
                    break
            else:
                check = 1
                recall_list[int(dfs[s]['count'].iloc[i])] =  dfs[s]['recall'].iloc[i]
                      
    for i in range(8):
        final_dict = {
            'story':story_list[i],
            'schema':schema_list[i],
            'recall':recall_list[i]
        }
        
        this_dict[i+1] = final_dict
        
    participants[this_p] = this_dict


In [85]:
participants

{'082819_p1': {1: {'story': '32', 'schema': 'Social', 'recall': 0.0},
  2: {'story': '21',
   'schema': 'Social',
   'recall': 'After nearly two years of dating, senna finally decided to propose to his girlfriend alvira. he chose a nice restaurant near the ocean as the setting. alvira said yes, and texted her father the good news.'},
  3: {'story': '12',
   'schema': 'Location',
   'recall': 'After spending time on vacation together in jamaica, jessie decided to tell calvin that she wanted to break up with him as they were trying to catch their flight home. the two agreed to part ways.'},
  4: {'story': '41',
   'schema': 'Location',
   'recall': 'James and teresa met at a sushi restaurant in atlanta after teresa ran into a glass door while texting on her phone. the two ended up sharing a meal together before james asked teresa to join him for another engagement.'},
  5: {'story': '14',
   'schema': 'Social',
   'recall': "Maria decided to tell henry that she thought it was best that t

In [86]:
#output each participant's recalls into a textfile, include the schema
#print just the stories into a text file


for key in participants:
    name = 'raw_free_recall_ebsFF_111919/' + key + '_free_recall'
    file = open(name,'w') 
    file.write('Participant'+ key) 
    file.write('\n\n')
    for i in range(8):
        file.write(participants[key][i+1]['story'])
        file.write('\n')
        file.write(participants[key][i+1]['schema'])
        file.write('\n')
        if participants[key][i+1]['recall'] == 0:
            file.write('0')
        else:
            file.write(participants[key][i+1]['recall'])
        file.write('\n\n\n\n\n')
    file.close()

In [87]:
#output each participant's recalls into a textfile, without the schema, for Tina
#print just the stories into a text file

for key in participants:
    name = 'tina_free_recall_pilotii/' + key 
    file = open(name,'w') 
    for i in range(8):
        file.write(participants[key][i+1]['story'])
        file.write('\n')
        if participants[key][i+1]['recall'] == 0:
            file.write('0')
        else:
            file.write(participants[key][i+1]['recall'])
        file.write('\n\n\n\n\n')
    file.close()

In [7]:
#Part 2: Export free recalls grouped by story
story_recalls = {}

for key in participants:
    for i in range(1,8):
        this_dict = {
            'participant' : key,
            'schema': participants[key][i]['schema'],
            'recall': participants[key][i]['recall'],
        }
        
        if participants[key][i]['story'] in story_recalls:
            story_recalls[participants[key][i]['story']] = np.vstack((story_recalls[participants[key][i]['story']], this_dict))
        else:
            story_recalls[participants[key][i]['story']] = this_dict

In [19]:
#make a dictionary of stories of participants with their schema priming
subj_schemas = {}

for key in story_recalls:
    subjs = {}
    for i in range(len(story_recalls[key])):
        subjs[story_recalls[key][i][0]['participant']] = story_recalls[key][i][0]['schema']
    subj_schemas[key] = subjs
        

In [23]:
#export the subj_schemas dict
pickle_out = open("subj_schemas.pickle","wb")
pickle.dump(subj_schemas, pickle_out)
pickle_out.close()

In [90]:
#print out each story
for key in story_recalls:
    name = 'raw_free_recall_ebsff_by_story/' + key + '.txt'
    file = open(name,'w') 
    for i in range(len(story_recalls[key])):
        file.write(story_recalls[key][i][0]['participant'])
        file.write('\n')
        file.write(story_recalls[key][i][0]['schema'])
        file.write('\n')
        if story_recalls[key][i][0]['recall'] == 0:
            file.write('0')
        else:
            file.write(story_recalls[key][i][0]['recall'])
        file.write('\n\n\n\n\n')
    file.close()

In [91]:
#Part 3: convert each set of story recalls into a scorable excel sheet

In [133]:
#step 0: import story stats in order to get story lengths
import pickle
pickle_in = open("story_stats.pickle","rb")
story_stats = pickle.load(pickle_in)

In [146]:
#make a pandas dataframe of each participant for each story\

for key in story_recalls:
    name = 'score_sheets_free_recall_ebsFF/' + key + '.xlsx'
    writer = pd.ExcelWriter(name, engine='xlsxwriter')
    for i in range(len(story_recalls[key])):
        if story_recalls[key][i][0]['recall'] == 0:
            continue
        else:
            #split the sentences of the recall
            sent = story_recalls[key][i][0]['recall'].split('.')
            #put all of the sentences into a dataframe, where each sentence is a new row
            df = pd.DataFrame({'sentences':sent})
            # add as many columns as there are sentences in each story
            for j in range(story_stats[key][3]):
                df[j+1] = None
            df.to_excel(writer, sheet_name = story_recalls[key][i][0]['participant'])
    writer.save()
            
            
        
            
            
            
            

In [11]:
story_recalls

{'32': array([[{'participant': '082819_p1', 'schema': 'Social', 'recall': 0.0}],
        [{'participant': '090419_p1', 'schema': 'Location', 'recall': 0.0}],
        [{'participant': '090419_p3', 'schema': 'Location', 'recall': 'Jeff and william are at an airport discussing business. the two met the other day at maybe a bar when they were discussing wines. coincidentally, jeff is a wine maker/owner and william distributes to large companies. the two are discussing details on potential sales while passing through security. initially william offers 800 dollars for a crate of samples to jeff, citing the amount he loses on these discounted wine bottles will be made up in potential contracts to large businesses. jeff initially nods, albeit weakly, but stands more firm after they pass through airport security. jeff is afraid of the potential losses after looking through the papers william gave him and renegotiates. in the end, jeff agrees to sell a 600 dollar shipment for 400 dollars to will